In [1]:
import mysql.connector as mysql
from mysql.connector import Error
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv("Airplane_Crashes_and_Fatalities_Since_1908.csv")

In [3]:
db_name='airplane_crashes'
try:
    mydb = mysql.connect(host='localhost', user='root', password='franci22') # you can add the auth_plugin here too (ref line 26)
    if mydb.is_connected():
        mycursor = mydb.cursor()
        mycursor.execute('SHOW DATABASES')
        result = mycursor.fetchall()
        print(result)
        for x in result:
            if db_name == x[0]:
                mycursor.execute('DROP DATABASE ' + db_name) # delete old database
                mydb.commit() # make the changes official
                print("The database already exists! The old database has been deleted!)")

        mycursor.execute("CREATE DATABASE "+ db_name)
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

[('electronics',), ('example_database',), ('GoShare',), ('information_schema',), ('mid_practice',), ('movies',), ('mysql',), ('parents',), ('performance_schema',), ('sys',)]
Database is created


In [4]:
mycursor.execute('USE' + ' ' + db_name)

In [5]:
#create tables
mycursor.execute(
      '''
        CREATE TABLE `Airplane` (
          `registration` varchar(100),
          `serial_number` varchar(100) NOT NULL,
          `operator` varchar(100),
          `type` varchar(100),
          PRIMARY KEY (registration, serial_number)
        );
      '''
      )

In [6]:
mycursor.execute(
      '''
        CREATE TABLE `Flight` (
           `flight_number` varchar(100),
           `aboard` int,
           `route` varchar(100),
           `serial_number` varchar(100),
           `registration` varchar(100),
           PRIMARY KEY (`flight_number`),
           FOREIGN KEY (`registration`, `serial_number`) REFERENCES `Airplane` (`registration`,`serial_number`)
        );
      '''
      )

In [7]:
mycursor.execute(
    '''
        CREATE TABLE `Crash` (
          `index` int,
          `location` varchar(100),
          `time` time,
          `date` date,
          `summary` varchar(100),
          `serial number` varchar(100),
          `registration` varchar(100),
          PRIMARY KEY (`index`),
          FOREIGN KEY (`registration`, `serial number`) REFERENCES `Airplane`(`registration`, `serial_number`)
        );
    '''
)

In [8]:
mycursor.execute(
    '''
    CREATE TABLE `Fatalities` (
    `number of fatalities` int,
    `ground` int,
    `crash index` int,
    PRIMARY KEY (`crash index`),
    FOREIGN KEY (`crash index`) REFERENCES `Crash`(`index`)
);
    '''
)

In [15]:
selection = dataset.iloc[3196:, 1:] #take values from 1980-90 and take all columns except from the first
null_val = selection.isnull().sum()
selection = selection.drop(labels='Flight #', axis=1)

In [16]:
import random
flight_number = []

while len(flight_number)!=2072:
    n = random.randint(1000, 10000)
    if n not in flight_number:
        flight_number.append(n)



dup = {x for x in flight_number if flight_number.count(x) >1}
#print(dup)
selection.insert(6,'Flight#', flight_number)
serial_num = selection.loc[3196:, 'cn/In']

#print(serial_num)


n = random.randint(100, 8000)
if n not in serial_num:
    selection['cn/In'].fillna(n, inplace = True)

serial_num = selection.loc[3196:, 'cn/In']
serial_num_ = list(serial_num)
dup2 = {i for i in serial_num_ if serial_num_.count(i) >1}

In [17]:
datairplane = selection[['cn/In', 'Registration', 'Operator', 'Type']]

In [18]:
datairplane.head(2000)

,cn/In,Registration,Operator,Type
3196,474,HH-CNC,Haïti Air Inter,Britten Norman BN Islander
3197,0,N442NC,Private charter,Cessna 441 Conquest
3198,19817/537,EP-IRD,Iran National Airlines,Boeing B-727-86
3199,45/3N,PK-PCX,Pelita Air Service,CASA 212 Aviocar 100
3200,545,5003,Military - Union of Burma Air Force,Fairchild FH-227H
...,...,...,...,...
5191,2570,PP-MUM,BHS - Brazilian Helicopter Service Air Taxi,Eurocopter AS332L2 Super Puma
5192,0,0,United Nations Mission,Mi-17
5193,UE-322,5N-JAH,Trade Wings Aviation Ltd.,Beechcraft 1900D
5194,1AJ007-17,PZ-TSO,Blue Wing Airlines,Antonov An-28


In [14]:
for i,row in datairplane.iterrows():
    sql = "INSERT INTO airplane_crashes.Airplane VALUES (%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['cn/In'], row['Registration'], row['Operator'], row['Type']]))
    mydb.commit()

IntegrityError: 1062 (23000): Duplicate entry '474-HH-CNC' for key 'airplane.PRIMARY'

In [ ]:
mycursor.execute("SELECT * FROM Airplane")
result = mycursor.fetchall()
print(result)

In [ ]:
dataflight = selection[['Flight#', 'Aboard', 'Route', 'cn/In', 'Registration']]

In [ ]:
for i, row in dataflight.iterrows():
    sql = "INSERT INTO airplane_crashes.Flight VALUES (%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['Flight#'], row['Aboard'], row['Route'], row['cn/In'], row['Registration']]))

    mydb.commit()